# Your first scraper
In this project, we will guide you step by step through the process of:

1. creating a self-contained development environment.
1. retrieving some information from an API (a website for computers)
2. leveraging it to scrape a website that does not provide an API
3. saving the output for later processing

Here we query an API for a list of countries and their past leaders. We then extract and sanitize their short bio from Wikipedia. Finally, we save the data to disk.

This task is often the first (coding) step of a datascience project and you will often come back to it in the future.

You will study topics such as *scraping*, *data structures*, *regular expressions*, *concurrency* and *file handling*. We will point out useful resources at the appropriate time. 

Let's dive in!

## 0. Creating a clean environment

Use the [`venv`](https://docs.python.org/3/library/venv.html) command to create a new environment called `wikipedia_scraper_env`.

Activate it and add it to you `.gitignore` file. 

You will find more info about virtual environments in the course content and on the web.

## 1. API Scraping

### 1a. A simple API query
You will start with the basics: how to do a simple request to an [API endpoint](../../2.python/2.python_advanced/05.Scraping/5.apis.ipynb).

You will use the [requests](https://requests.readthedocs.io/en/latest/) external library through the `import` keyword. NOTE: external libraries need to be installed first. Check the [request Quickstart](https://requests.readthedocs.io/en/latest/user/quickstart/) section of the documentation to:

1. Use the `get()` method to connect to this endpoint: https://country-leaders.onrender.com/status
2. Check if the `status_code` is equal to 200, which means OK.
    * if OK, `print()` the `text`` of the response.
    * if not, `print()` the `status_code`. 

Here is an explanation of [HTTP status codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes).


In [1]:
# import the requests library
import requests

# assign the root url (without /status) to the root_url variable
root_url = "https://country-leaders.onrender.com"

# assign the /status endpoint to another variable called status_url
status_url = f"{root_url}/status"

# query the /status endpoint using the get() method and store it in the req variable
req = requests.get(status_url)

# check the status_code using a condition and print appropriate messages
if req.status_code == 200:
    print(req.text)   # print the text of the response
else:
    print(req.status_code)  # print the status code if not OK



"Alive"


### 1b. Dealing with JSON

[JSON](https://quickref.me/json) is the preferred format to deal with data over the web. You cannot avoid it so you would better get acquainted.

Connect to another endpoint called `/countries` but this time the API will return data in the JSON format. 


In [2]:
# Set the countries_url variable (1 line)
countries_url = f"{root_url}/countries"

# query the /countries endpoint using the get() method and store it in the req variable (1 line)
req = requests.get(countries_url)

# Get the JSON content and store it in the countries variable (1 line)
countries = req.json()

# display the request's status code and the countries variable (1 line)
print(req.status_code)
print(countries)


403
{'message': 'The cookie is missing'}


### 1c. Cookies anyone?

It looks like the access to this API is restricted...
Query the `/cookie` endpoint and extract the appropriate field to access your cookie.

You will need to use this cookie in each of the following API requests.

In [3]:
# Set the cookie_url variable
cookie_url = f"{root_url}/cookie"

# Query the enpoint, set the cookies variable and display it (2 lines)
req = requests.get(cookie_url)
cookies = req.cookies
cookies


<RequestsCookieJar[Cookie(version=0, name='user_cookie', value='863c0fcc-5344-43ec-ac9f-e8358f8cecd5', port=None, port_specified=False, domain='country-leaders.onrender.com', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=False, expires=None, discard=True, comment=None, comment_url=None, rest={'SameSite': 'lax'}, rfc2109=False)]>

Try to query the countries endpoint using the cookie, save the output and print it.

In [4]:
# query the /countries endpoint, assign the output to the countries variable (1 line)
countries = requests.get(countries_url, cookies=cookies).json()                            
# Here cookies=cookies means the above generated cookie is send to root url to get the cookies from web, the server then recognizes your session and return 

print(countries)



['be', 'us', 'ru', 'fr', 'ma']


Chances are the cookie has expired... Thanksfully, you got a nice error message. For now, simply execute the last 2 cells quickly so you get a result.

### 1d. Getting the actual data from the API

Query the `/leaders` endpoint.

In [5]:
# Set the leaders_url variable (1 line)
leader_url = f"{root_url}/leaders"

# query the /leaders endpoint, assign the output to the leaders variable (1 line)
leaders = requests.get(leader_url, cookies=cookies).json()

# display the leaders variable (1 line)
print(leaders)

{'message': 'Please specify a country'}


It looks like this endpoint requires additional information in order to return its result. Check the API [*documentation*](https://country-leaders.onrender.com/docs) in your web browser.

Change the query to accept *parameters*. You should know where to find help by now.

In [6]:
# query the /leaders endpoint using cookies and parameters (take any country in countries)
params ={"country": country}
# assign the output to the leaders variable (1 line)
leaders = requests.get(leader_url, cookies=cookies, params=params).json()
# display the leaders variable (1 line)
print(leaders)


NameError: name 'country' is not defined

### 1e. A sneak peak at the data (finally)

Look inside a few examples. Notice the dictionary keys available for each entry. You have your first example of *structured data*. This data was sanitized for your benefit, meaning it is readily exploitable without modification.

You will also notice there is a Wikipedia link for each entry. You will need to extract additional information there. This will be a case of *semi-structured* data.

The /countries endpoint returns a `list` of several country codes.

You need to loop through this list and query the /leaders endpoint for each one. Save each `json` result in a dictionary called `leaders_per_country`.

In [ ]:
# 4 lines
leader_per_country = {}

for country in countries:
    leader_per_country[country] = requests.get(leader_url, cookies=cookies, params=params).json()
print(leader_per_country)


    
    


{'be': [{'id': 'Q57553', 'first_name': 'Mohammed', 'last_name': 'None', 'birth_date': '1963-08-21', 'death_date': None, 'place_of_birth': 'Rabat', 'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%B3%D8%A7%D8%AF%D8%B3_%D8%A8%D9%86_%D8%A7%D9%84%D8%AD%D8%B3%D9%86', 'start_mandate': '1999-07-23', 'end_mandate': None}, {'id': 'Q69103', 'first_name': 'Hassan', 'last_name': 'None', 'birth_date': '1929-07-09', 'death_date': '1999-07-20', 'place_of_birth': 'Rabat', 'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B3%D9%86_%D8%A7%D9%84%D8%AB%D8%A7%D9%86%D9%8A_%D8%A8%D9%86_%D9%85%D8%AD%D9%85%D8%AF', 'start_mandate': '1961-02-26', 'end_mandate': '1999-07-23'}, {'id': 'Q193874', 'first_name': 'Mohammed', 'last_name': 'None', 'birth_date': '1909-08-10', 'death_date': '1961-02-26', 'place_of_birth': 'Fez', 'wikipedia_url': 'https://ar.wikipedia.org/wiki/%D9%85%D8%AD%D9%85%D8%AF_%D8%A7%D9%84%D8%AE%D8%A7%D9%85%D8%B3_%D8%A8%D9%86_%D9%8A%D9%88%

In [ ]:
# or 1 line


It is finally time to create a `get_leaders()` function for the above code. You will build on it later-on. This function takes no parameter. Inside it, you will need to:
1. define the urls
2. get the cookies
2. get the countries
3. loop over them and save their leaders in a dictionary
4. return the dictionary

In [ ]:
# < 15 lines
import requests

def get_leader():
    # 1. define the urls 
    root_url = "https://country-leaders.onrender.com"
    cookie_url = f"{root_url}/cookie"
    countries_url = f"{root_url}/countries"
    leader_url = f"{root_url}/leaders"

    # 2. get the cookies 
    req = requests.get(cookie_url)
    cookies = req.cookies
    cookies

    # 3. get the countries 
    countries = requests.get(countries_url, cookies=cookies).json() 

    # 4. loop over countries and collect leaders
    leader_per_country = {}

    for country in countries:
        params = {"country": country}
        leader_per_country[country] = requests.get(leader_url, cookies=cookies, params=params).json()
    return leader_per_country
    
    

    
    
    
    


Test your function, save the result in the `leaders_per_country` dictionary and check its ouput.

In [ ]:
# 2 lines
leader_per_country = get_leader()

print(leader_per_country["be"])



[{'id': 'Q12978', 'first_name': 'Guy', 'last_name': 'Verhofstadt', 'birth_date': '1953-04-11', 'death_date': None, 'place_of_birth': 'Dendermonde', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Guy_Verhofstadt', 'start_mandate': '1999-07-12', 'end_mandate': '2008-03-20'}, {'id': 'Q12981', 'first_name': 'Yves', 'last_name': 'Leterme', 'birth_date': '1960-10-06', 'death_date': None, 'place_of_birth': 'Wervik', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Yves_Leterme', 'start_mandate': '2009-11-25', 'end_mandate': '2011-12-06'}, {'id': 'Q12983', 'first_name': 'Herman', 'last_name': 'None', 'birth_date': '1947-10-31', 'death_date': None, 'place_of_birth': 'Etterbeek', 'wikipedia_url': 'https://nl.wikipedia.org/wiki/Herman_Van_Rompuy', 'start_mandate': '2008-12-30', 'end_mandate': '2009-11-25'}, {'id': 'Q14989', 'first_name': 'Léon', 'last_name': 'Delacroix', 'birth_date': '1867-12-27', 'death_date': '1929-10-15', 'place_of_birth': 'Saint-Josse-ten-Noode', 'wikipedia_url': 'https://nl

## 2. Extracting data from Wikipedia

Query one of the leaders' Wikipedia urls and display its `text` (not JSON).

In [ ]:
# 3 lines


Ouch! You get the raw HTML code of the webpage. If you try to deal with it without tools, you will be there all night. Instead, use the [beautiful soup 4](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) *external* library. You will find more info about it [here](../../2.python/2.python_advanced/05.Scraping/1.beautifulsoup_basic.ipynb) and [here](../../2.python/2.python_advanced/05.Scraping/2.beautifulsoup_advanced.ipynb)

Using the Quickstart section, start by importing the library and loading the output of your `get_text()` function.

Use the `prettify()` function and print it to take a look. You will start the actual parsing in the next step.

In [ ]:
# 3 lines
from bs4 import BeautifulSoup
import requests

def get_text():
    url = "https://nl.wikipedia.org/wiki/Guy_Verhofstadt"
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    print(soup.prettify())

# soup.prettify() is a BeautifulSoup method that formats the HTML nicely: Adds proper indentation for nested tags. 
# Makes it easier to read and debug


That looks better but you need to extract the right part of the webpage: the text of the first paragraph.

It is a bit tricky because Wikipedia pages slightly differ in structure from one language to the next. We cannot simply get the text for the first HTML paragraph.

You will start by getting all the HTML paragraphs from the HTML source and saving them in the `paragraphs` variable.

Use the documentation or google the appropriate keywords.

In [ ]:
# 2 lines

for p in soup.find_all("p"):
    print(p.text)
   



Guy Maurice Marie-Louise Verhofstadt[5] (uitspraakⓘ) (Dendermonde, 11 april 1953) is een Belgisch politicus voor de Open Vlaamse Liberalen en Democraten (Open Vld). Hij was premier van België van 12 juli 1999 tot 20 maart 2008 in drie regeringen. Hij beëindigde zijn actieve politieke carrière in het Europees Parlement, waar hij van 2009 tot 2019 fractieleider van de Alliantie van Liberalen en Democraten voor Europa (ALDE) was.[6]

Na zijn middelbareschooltijd, die hij doorbracht aan het Koninklijk Atheneum Voskenslaan, ging hij rechten studeren aan de Rijksuniversiteit Gent, waar hij in 1975 afstudeerde als licentiaat. Nadien werd hij tot in 1994 advocaat aan de Balie van Gent.[7]

In zijn studentenperiode in Gent begon hij zijn politieke carrière, allereerst in het Liberaal Vlaams Studentenverbond, waarvan hij van 1972 tot 1974 de voorzitter was. Later kwam hij in contact met Willy De Clercq, een kopstuk van de PVV in Gent, en hij kon als diens pupil opklimmen in de partij. Van 1977 t

If you try different urls, you might find that the paragraph you want may be at a different index each time.

That is where you need to be clever and ask yourself what would be a reliable way to identify the right index ie. which string matches only the first paragraph whatever the language...

Spend a good 30 minutes on the problem and brainstorm with your fellow learners. If you come out empty handed, ask your coach.

1. Loop over the HTML paragraphs
2. When you have identified the correct one:
   * Store the [text](https://www.crummy.com/software/BeautifulSoup/bs4/doc/#output) inside the `first_paragraph` variable
   * Exit the loop

In [ ]:
# <10 lines


# Loop through paragraphs and find the first non-empty one
# for p in soup.find_all("p"):  # 1️.loop through each <p> element
#     text = p.get_text(strip=True)  # extract the paragraph text, remove extra spaces
#     if text:
#         first_paragraph = text
#         break


first_paragraph = None  # initialize variable

    # Loop through all paragraphs
url = "https://nl.wikipedia.org/wiki/Guy_Verhofstadt"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"} 
r = requests.get(url, headers = headers)
soup = BeautifulSoup(r.content, "html.parser")
for p in soup.find_all("p"):
    text = p.get_text(strip=True)  # extract text and remove extra spaces

        # Take the first paragraph directly
    first_paragraph = text
    break  # exit loop once found

print(soup)



<!DOCTYPE html>

<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-enabled skin-theme-clientpref-day vector-sticky-header-enabled vector-toc-available" dir="ltr" lang="nl">
<head>
<meta charset="utf-8"/>
<title>Guy Verhofstadt - Wikipedia</title>
<script>(function(){var className="client-js vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-f

At this stage, you can create a function to maintain consistency in your code. We will give you its *skeleton*, you will copy the code you wrote and make it work inside a function.

Don't forget to test your function.

In [ ]:
# 10 lines
# def get_first_paragraph(wikipedia_url):
#   print(wikipedia_url) # keep this for the rest of the notebook
#   [insert your code]
#   return first_paragraph




from bs4 import BeautifulSoup
import requests

def get_first_paragraph(wikipedia_url, min_length=200):
    print(wikipedia_url)  # track which page is being processed
    
    # Use headers to mimic a real browser
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
    r = requests.get(wikipedia_url, headers=headers)
    soup = BeautifulSoup(r.content, "html.parser")
    
    first_paragraph = None  # initialize variable

    # Loop through all paragraphs and take the first one
    for p in soup.find_all("p"):
        text = p.get_text(strip=True)  # extract text and remove extra spaces
        # Only accept paragraphs with minimum length
        if len(text) >= min_length:
            first_paragraph = text
            break  # Stop at first paragraph that meets requirement
    
    return first_paragraph













# wikipedia_url → input URL for the leader’s page.
# requests.get() → downloads the HTML.
# BeautifulSoup(..., "html.parser") → parses the HTML.
# Loop over all <p> tags and skip empty paragraphs.
# first_paragraph stores the first meaningful paragraph.




In [ ]:
# Test: 3 lines
url = "https://fr.wikipedia.org/wiki/Napol%C3%A9on_III"

intro = get_first_paragraph(url)
print(intro)





https://fr.wikipedia.org/wiki/Napol%C3%A9on_III
Louis-Napoléon Bonaparte, né le20 avril 1808àParis(France) et mort le9 janvier 1873àChislehurst(Royaume-Uni), est unmonarqueethomme d'Étatfrançais. Il est à la fois l'uniqueprésidentde laDeuxième République, le premierchef d'État françaisélu ausuffrage universel masculinle10 décembre 1848, le premierprésident de la République françaiseet, après la proclamation de l'Empirele2 décembre 1852, le derniermonarquedu pays en tant qu'empereur des Françaissous le nom deNapoléonIII.


### 2a. Regular expressions to the rescue

Now that you have extracted the content of the first paragraph, the only thing that remains to finish your Wikipedia scraper is to sanitize the output.

Indeed some Wikipedia references, HTML code, phonetic pronunciation etc. may linger. You might find *regular expressions* handy to get rid of them and obtain pristine text. You will find some useful documentation about regular expressions [here](../../2.python/2.python_advanced/03.Regex/regex.ipynb)

Once you have one of your regex working online, try it in the cell below. 

Hints: 
* Check the `sub()` method documentation.
* Make sure to test urls in different languages. Some may look good but other do not.

In [ ]:
# 3 lines



Overwrite the `get_first_paragraph()` function by applying your regex to the first paragraph before returning it.

In [ ]:
# 10 lines


Come up with other regexes to capture other patterns and sanitize the outputs completely. Modify your `get_first_paragraph()` function accordingly.

In [ ]:
# < 20 lines


## 3. Putting it all together

Let's go back to your `get_leaders()` function and update it with an *inner* loop over each leader. You will query the url provided and extract the first paragraph using the `get_first_paragraph()` function you just finished. You will then update that `leader`'s dictionary and move on to the next one.

Notice, the rest of the code should not change since you modify the leader's data one by one.

In [ ]:
# < 20 lines

from bs4 import BeautifulSoup
import requests

def get_first_paragraph(wikipedia_url, min_length=200):
    print(wikipedia_url)  # track which page is being processed
    
    # Use headers to mimic a real browser
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
    r = requests.get(wikipedia_url, headers=headers)
    soup = BeautifulSoup(r.content, "html.parser")
    
    first_paragraph = None  # initialize variable

    # Loop through all paragraphs and take the first one
    for p in soup.find_all("p"):
        text = p.get_text(strip=True)  # extract text and remove extra spaces
        # Only accept paragraphs with minimum length
        if len(text) >= min_length:
            first_paragraph = text
            break  # Stop at first paragraph that meets requirement
    
    return first_paragraph

# Function to get leaders and add first paragraph
def get_leader():
    # 1. define the urls 
    root_url = "https://country-leaders.onrender.com"
    cookie_url = f"{root_url}/cookie"
    countries_url = f"{root_url}/countries"
    leader_url = f"{root_url}/leaders"

    # 2. get the cookies 
    req = requests.get(cookie_url)
    cookies = req.cookies

    # 3. get the countries 
    countries = requests.get(countries_url, cookies=cookies).json() 

    # 4. loop over countries and collect leaders
    leader_per_country = {}

    for country in countries:
        params = {"country": country}
        leaders = requests.get(leader_url, cookies=cookies, params=params).json()
        
        # loop over each leader to add Wikipedia intro paragraph
        for leader in leaders:
            if "wikipedia_url" in leader:  # make sure the key exists
                wiki_url = leader["wikipedia_url"]
                leader_intro = get_first_paragraph(wiki_url)
                leader["intro_paragraph"] = leader_intro  # add intro to leader dict
        
        leader_per_country[country] = leaders

    return leader_per_country



In [ ]:
# Check the output of your function (2 lines)
leaders_data = get_leader()
print(leaders_data["ma"][0]["intro_paragraph"])


https://nl.wikipedia.org/wiki/Guy_Verhofstadt
https://nl.wikipedia.org/wiki/Yves_Leterme
https://nl.wikipedia.org/wiki/Herman_Van_Rompuy
https://nl.wikipedia.org/wiki/L%C3%A9on_Delacroix
https://nl.wikipedia.org/wiki/Henri_Carton_de_Wiart
https://nl.wikipedia.org/wiki/Georges_Theunis
https://nl.wikipedia.org/wiki/Aloys_Van_de_Vyvere
https://nl.wikipedia.org/wiki/Prosper_Poullet
https://nl.wikipedia.org/wiki/Henri_Jaspar
https://nl.wikipedia.org/wiki/Jules_Renkin
https://nl.wikipedia.org/wiki/Paul_van_Zeeland
https://nl.wikipedia.org/wiki/Achiel_Van_Acker
https://nl.wikipedia.org/wiki/Camille_Huysmans
https://nl.wikipedia.org/wiki/Gaston_Eyskens
https://nl.wikipedia.org/wiki/Leo_Tindemans
https://nl.wikipedia.org/wiki/Elio_Di_Rupo
https://nl.wikipedia.org/wiki/Jean-Luc_Dehaene
https://nl.wikipedia.org/wiki/Frans_Schollaert
https://nl.wikipedia.org/wiki/Pieter_de_Decker
https://nl.wikipedia.org/wiki/Paul-Henri_Spaak
https://nl.wikipedia.org/wiki/Auguste_Beernaert
https://nl.wikipedia.org

KeyError: 0

Does the function crash in the middle of the loop? Chances are the cookies have expired while looping over the leaders.

Modify your function with an *exception* or check if the `status_code` is a cookie error. In either case, get new cookies and query the api again.

If your code did not crash,

In [ ]:
# STEP 1: Import all necessary libraries at the TOP
from bs4 import BeautifulSoup
import requests

# STEP 2: Define get_first_paragraph FIRST (before get_leader, since get_leader calls it)
def get_first_paragraph(wikipedia_url, min_length=100):
    print(wikipedia_url)  # track which page is being processed
    # Use headers to mimic a real browser
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
    r = requests.get(wikipedia_url, headers=headers)
    soup = BeautifulSoup(r.content, "html.parser")
    
    first_paragraph = None  # initialize variable
    
    for p in soup.find_all("p"):
        text = p.get_text(strip=True)
        
        # Only accept paragraphs with minimum length
        if len(text) >= min_length:
            first_paragraph = text
            break  # Stop at first paragraph that meets requirement
    
    return first_paragraph

# STEP 3: Define get_leader function
def get_leader():
    # 1. define the urls 
    root_url = "https://country-leaders.onrender.com"
    cookie_url = f"{root_url}/cookie"
    countries_url = f"{root_url}/countries"
    leader_url = f"{root_url}/leaders"
    
    # 2. get the cookies 
    req = requests.get(cookie_url)
    cookies = req.cookies
    
    # 3. get the countries 
    response = requests.get(countries_url, cookies=cookies)
    if response.status_code != 200:
        req = requests.get(cookie_url)
        cookies = req.cookies
        response = requests.get(countries_url, cookies=cookies)
    countries = response.json()
    
    # 4. loop over countries and collect leaders
    leader_per_country = {}
    for country in countries:
        params = {"country": country}
        response = requests.get(leader_url, cookies=cookies, params=params)
        
        # Check if cookies expired
        if response.status_code != 200:
            # Get new cookies
            req = requests.get(cookie_url)
            cookies = req.cookies
            # Try the request again with fresh cookies
            response = requests.get(leader_url, cookies=cookies, params=params)
        
        # Now convert to JSON
        leaders = response.json()
        
        # loop over each leader to add Wikipedia intro paragraph
        for leader in leaders:
            if "wikipedia_url" in leader:
                wiki_url = leader["wikipedia_url"]
                leader_intro = get_first_paragraph(wiki_url)
                leader["intro_paragraph"] = leader_intro
        
        leader_per_country[country] = leaders
    
    return leader_per_country

Check the output of your function again.

In [ ]:
# Check the output of your function (1 line)
print(get_leader())

https://nl.wikipedia.org/wiki/Guy_Verhofstadt
https://nl.wikipedia.org/wiki/Yves_Leterme
https://nl.wikipedia.org/wiki/Herman_Van_Rompuy
https://nl.wikipedia.org/wiki/L%C3%A9on_Delacroix
https://nl.wikipedia.org/wiki/Henri_Carton_de_Wiart
https://nl.wikipedia.org/wiki/Georges_Theunis
https://nl.wikipedia.org/wiki/Aloys_Van_de_Vyvere
https://nl.wikipedia.org/wiki/Prosper_Poullet
https://nl.wikipedia.org/wiki/Henri_Jaspar
https://nl.wikipedia.org/wiki/Jules_Renkin
https://nl.wikipedia.org/wiki/Paul_van_Zeeland
https://nl.wikipedia.org/wiki/Achiel_Van_Acker
https://nl.wikipedia.org/wiki/Camille_Huysmans
https://nl.wikipedia.org/wiki/Gaston_Eyskens
https://nl.wikipedia.org/wiki/Leo_Tindemans
https://nl.wikipedia.org/wiki/Elio_Di_Rupo
https://nl.wikipedia.org/wiki/Jean-Luc_Dehaene
https://nl.wikipedia.org/wiki/Frans_Schollaert
https://nl.wikipedia.org/wiki/Pieter_de_Decker
https://nl.wikipedia.org/wiki/Paul-Henri_Spaak
https://nl.wikipedia.org/wiki/Auguste_Beernaert
https://nl.wikipedia.org

Well done! It took a while however... Let's speed things up. The main *bottleneck* is the loop. We call on the Wikipedia website many times.

You will use the same *session* to call all the wikipedia pages. Check the *Advanced Usage* section of the Requests module's documentation.

Start by modifying the `get_first_paragraph()` function to accept a session parameter and adjust the `get()` method call.

In [23]:
# STEP 1: Import all necessary libraries at the TOP
from bs4 import BeautifulSoup
import requests

# STEP 2: Define get_first_paragraph FIRST (before get_leader, since get_leader calls it)
def get_first_paragraph(wikipedia_url, session, min_length=200):
    # Use headers to mimic a real browser
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
    r = session.get(wikipedia_url, headers=headers)
    soup = BeautifulSoup(r.content, "html.parser")
    
    first_paragraph = None  # initialize variable
    
    # Loop through all paragraphs and take the first one
    for p in soup.find_all("p"):
        text = p.get_text(strip=True)
        
        # Only accept paragraphs with minimum length
        if len(text) >= min_length:
            first_paragraph = text
            break  # Stop at first paragraph that meets requirement
    
    return first_paragraph

# STEP 3: Define get_leader function
def get_leader():
    # 1. define the urls 
    root_url = "https://country-leaders.onrender.com"
    cookie_url = f"{root_url}/cookie"
    countries_url = f"{root_url}/countries"
    leader_url = f"{root_url}/leaders"
    
    # 2. get the cookies 
    req = requests.get(cookie_url)
    cookies = req.cookies
    session = requests.Session()
    # 3. get the countries 
    response = requests.get(countries_url, cookies=cookies)
    if response.status_code != 200:
        req = requests.get(cookie_url)
        cookies = req.cookies
        response = requests.get(countries_url, cookies=cookies)
    countries = response.json()
    
    # 4. loop over countries and collect leaders
    leader_per_country = {}
    for country in countries:
        params = {"country": country}
        response = requests.get(leader_url, cookies=cookies, params=params)
        
        # Check if cookies expired
        if response.status_code != 200:
            # Get new cookies
            req = requests.get(cookie_url)
            cookies = req.cookies
            # Try the request again with fresh cookies
            response = requests.get(leader_url, cookies=cookies, params=params)
        
        # Now convert to JSON
        leaders = response.json()
        
        # loop over each leader to add Wikipedia intro paragraph
        for leader in leaders:
            if "wikipedia_url" in leader:
                wiki_url = leader["wikipedia_url"]
                leader_intro = get_first_paragraph(wiki_url,session)
                leader["intro_paragraph"] = leader_intro
        
        leader_per_country[country] = leaders
    
    return leader_per_country

Modify your `get_leaders()` function to make use of a single session for all the Wikipedia calls.
1. create a `Session` object outside of the loop over countries.
2. pass it to the `get_first_paragraph()` function as an argument.

In [ ]:
# STEP 1: Import all necessary libraries at the TOP
from bs4 import BeautifulSoup
import requests

# STEP 2: Define get_first_paragraph FIRST (before get_leader, since get_leader calls it)
def get_first_paragraph(wikipedia_url, session, min_length=200):
    # Use headers to mimic a real browser
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
    try:
        r = session.get(wikipedia_url, headers=headers, timeout=10)
        r.raise_for_status()
    except requests.RequestException:
        return None 
    
    soup = BeautifulSoup(r.content, "html.parser")
    
    # Prefer article body if present
    content = soup.find(id="mw-content-text")
    paragraphs = content.find_all("p") if content else soup.find_all("p")

    for p in paragraphs:
        text = p.get_text(" ", strip=True)
        # Only accept paragraphs with minimum length 
        if len(text) >= min_length and "may refer to:" not in text.lower():
            return text

    return None  # nothing suitable found

# STEP 3: Define get_leader function
def get_leader():
    # 1. define the urls 
    root_url = "https://country-leaders.onrender.com"
    cookie_url = f"{root_url}/cookie"
    countries_url = f"{root_url}/countries"
    leader_url = f"{root_url}/leaders"
    
   # single session for everything
    session = requests.Session()
    session.headers.update({"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"})

    # 2. get the cookies (seed session cookie)
    try:
        req = session.get(cookie_url, timeout=10)
        req.raise_for_status()
        cookies = req.cookies  # keep your variable name
    except requests.RequestException as e:
        print(f"⚠️ Error while fetching cookie: {e}")
        return {}

    # 3. get the countries 
    try:
        response = session.get(countries_url, timeout=10)
        if response.status_code == 403:  # cookie might be stale → refresh once
            req = session.get(cookie_url, timeout=10)
            response = session.get(countries_url, timeout=10)
        response.raise_for_status()
        countries = response.json()
    except Exception as e:
        print(f"⚠️ Error while fetching countries: {e}")
        return {}  # stop early or return empty result

    # 4. loop over countries and collect leaders
    leader_per_country = {}
    for country in countries:
        params = {"country": country}
        try:
            response = session.get(leader_url, params=params, timeout=10)
            if response.status_code == 403:
                req = session.get(cookie_url, timeout=10)
                response = session.get(leader_url, params=params, timeout=10)
            response.raise_for_status()
            leaders = response.json()
        except requests.RequestException as e:
            print(f"⚠️ Error fetching leaders for {country}: {e}")
            leaders = []
        except ValueError:
            leaders = []

        # loop over each leader to add Wikipedia intro paragraph
        for leader in leaders:
            if "wikipedia_url" in leader and leader["wikipedia_url"]:
                wiki_url = leader["wikipedia_url"]
                leader_intro = get_first_paragraph(wiki_url, session)
                leader["intro_paragraph"] = leader_intro

        leader_per_country[country] = leaders

    return leader_per_country



Test your new functions.



In [22]:
# interactive test
result = get_leader()

# Show available countries
print("Available countries:", list(result.keys()))
print()

# Let user choose a country
country_code = input("Enter country code ('be', 'fr', 'us', 'ru', 'ma'): ")

if country_code in result:
    leaders = result[country_code]
    print(f"\n{country_code.upper()} has {len(leaders)} leaders\n")
    
    # Show all leader names
    for i, leader in enumerate(leaders):
        print(f"{i+1}. {leader.get('first_name')} {leader.get('last_name')}")
    
    # Let user choose a leader
    leader_num = int(input("\nEnter leader number: ")) - 1
    
    if 0 <= leader_num < len(leaders):
        chosen = leaders[leader_num]
        print(f"\n{'='*60}")
        print(f"Name: {chosen.get('first_name')} {chosen.get('last_name')}")
        print(f"Birth: {chosen.get('birth_date')}")
        print(f"Term: {chosen.get('start_mandate')} to {chosen.get('end_mandate')}")
        print(f"\nIntro:\n{chosen.get('intro_paragraph')}")
    else:
        print("Invalid leader number!")
else:
    print("Country not found!")

Available countries: ['be', 'us', 'ru', 'fr', 'ma']


BE has 53 leaders

1. Guy Verhofstadt
2. Yves Leterme
3. Herman None
4. Léon Delacroix
5. Henry Carton
6. Georges Theunis
7. Aloïs Van de Vyvere
8. Prosper Poullet
9. Henri Jaspar
10. Jules None
11. Paul Van Zeeland
12. Achille Van Acker
13. Camille Huysmans
14. Gaston Eyskens
15. Leo Tindemans
16. Elio Di Rupo
17. Jean-Luc Dehaene
18. Frans Schollaert
19. Pieter Decker
20. Paul-Henri Spaak
21. Auguste Beernaert
22. Théodore Lefèvre
23. Paul None
24. Étienne Gerlache
25. Wilfried Martens
26. Mark Eyskens
27. Pierre Harmel
28. Hubert Pierlot
29. Jean-Baptiste Nothomb
30. Edmond None
31. Jean None
32. Joseph Lebeau
33. Charles Rogier
34. Barthélemy None
35. Felix None
36. Alexander De Croo
37. Henri de Brouckère
38. Albert None
39. Sylvain Weyer
40. Jules Burlet
41. Paul-Émile None
42. Jules Malou
43. Charles Broqueville
44. Joseph Pholien
45. Jean Van Houtte
46. Walthère Frère
47. Gérard Cooreman
48. Jules Vandenpeereboom
49. Paul No

## 4. Saving your hard work

The final step is to save the ``leaders_per_country`` dictionary in the `leaders.json` file using the [json](https://docs.python.org/3/library/json.html) module. Check out the `with` statement.

In [24]:
import json

leaders_data = get_leader()

with open('leaders.json', 'w', encoding='utf-8') as json_file:
    json.dump(leaders_data, json_file, indent=4, ensure_ascii=False)

print("Data saved to leaders.json successfully!")

# encoding='utf-8': Tells Python to use UTF-8 encoding
# ensure_ascii=False: Keeps original characters instead of escaping them

Data saved to leaders.json successfully!


Make sure the file can be read back. Write the code to read the file. And check the variables are the same.

In [12]:
# 3 lines

loaded_data = json.load(open('leaders.json', 'r', encoding='utf-8'))
original_data = get_leader()
print("Data matches!" if loaded_data == original_data else "Data differs!")

Data matches!


Make a function `save(leaders_per_country)` to call this code easily.

In [13]:
# 3 lines

def save(leaders_per_country):
    """Save leaders data to JSON file"""

    with open('leaders.json', 'w', encoding='utf-8') as f:
        json.dump(leaders_per_country, f, indent=4, ensure_ascii=False)
    
    print("Data saved to leaders.json successfully!")

In [14]:
# Call the function
# Get data
leaders_data = get_leader()
# Save using your new function
save(leaders_data)

Data saved to leaders.json successfully!


## 5. Tidy things up in a stand-alone python script

Congratulations! You now have a working scraper! However, your code is scattered throughout this notebook along side the tutorials. Hardly production ready...

Copy and paste what you need in a separate `leaders_scraper.py` file.
Make sure it works by calling `python3 leaders_scraper.py`

## (Optional) To go further

If you want to practice scraping, you can read this section and tackle the exercises.

1. Restructure your code by using OOP (see ReadMe).
2. You have noticed the API returns very partial results for country leaders. Many are missing. Overwrite the `get_leaders()` function to get its list from Wikipedia and extract their *personal details* from the frame on the side.

Good luck!